In [1]:
# Use GAN to generate synthesized characters on MNIST dataset

In [2]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image
from torchsummary import summary as PyTorchSummary

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [4]:
latent_size = 8
num_epochs = 200
batch_size = 100
sample_dir = 'samples2'

In [5]:
# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [6]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean = (0.5,0.5,0.5),
                                     std = (0.5, 0.5, 0.5))])

In [7]:
# MNIST dataset

In [8]:
mnist = torchvision.datasets.MNIST(root='./../data',
                                          train = True,
                                          transform = transform,
                                          download = True)

In [9]:
data_loader = torch.utils.data.DataLoader(dataset = mnist,
                                          batch_size = batch_size,
                                          shuffle = True)

## Build model

In [10]:
image_size = (1,28,28)

In [11]:
# Discriminator: a simple classification model
#Input: (1,28,28) image
#output: 0 or 1 (true of fake image)

In [12]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=5, stride = 1), #1x28x28->8x24x24
            nn.BatchNorm2d(8),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2))#8x12x12
        self.layer2 = nn.Sequential(
            nn.Conv2d(8, 16, kernel_size=5, stride = 1), #16x8x8
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2)) #16x4x4
        self.layer3 = nn.Sequential(
            nn.Conv2d(16, 64, kernel_size=3, stride = 1), #64x2x2
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, stride = 2)) #64x1x1
        self.fc = nn.Sequential(
            nn.Linear(64, 1),
            nn.Sigmoid())
    def forward(self,x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

In [13]:
# Generator: an upsample model
#Input: (latent size,) vector
#output: (1,28,28) fake image

In [14]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.upsample = nn.Sequential(
            nn.ConvTranspose2d(latent_size, 32, 5), #64x1x1 -> 32x5x5
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 5, stride = 2), #-> 16x13x13
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.ConvTranspose2d(16, 4, 3, stride = 2), #-> 4x27x27
            nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.ConvTranspose2d(4, 1, 2), #-> 1x28x28
            nn.Tanh())
        
    def forward(self,x):
        out = x.reshape(x.size(0),x.size(1),1,1)
        out = self.upsample(out)
        return out

In [15]:
D = Discriminator().to(device)
G = Generator().to(device)

In [16]:
PyTorchSummary(D, input_size = (1,28,28))
PyTorchSummary(G, input_size = (latent_size,1))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 24, 24]             208
       BatchNorm2d-2            [-1, 8, 24, 24]              16
              ReLU-3            [-1, 8, 24, 24]               0
         MaxPool2d-4            [-1, 8, 12, 12]               0
            Conv2d-5             [-1, 16, 8, 8]           3,216
       BatchNorm2d-6             [-1, 16, 8, 8]              32
              ReLU-7             [-1, 16, 8, 8]               0
         MaxPool2d-8             [-1, 16, 4, 4]               0
            Conv2d-9             [-1, 64, 2, 2]           9,280
      BatchNorm2d-10             [-1, 64, 2, 2]             128
             ReLU-11             [-1, 64, 2, 2]               0
        MaxPool2d-12             [-1, 64, 1, 1]               0
           Linear-13                    [-1, 1]              65
          Sigmoid-14                   

In [17]:
#Check Discriminator shape transform
x = torch.randn(100,1,28,28)
D(x).shape

torch.Size([100, 1])

In [18]:
#Check Generator shape transform
x = torch.randn(100,latent_size)
G(x).shape

torch.Size([100, 1, 28, 28])

In [19]:
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0004)

In [20]:
def denorm(x):
    out = (x+1)/2
    return out.clamp(0, 1)

In [21]:
def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

## Training

In [22]:
total_step = len(data_loader)

In [23]:
for epoch in range(num_epochs):
    for i, (images,_) in enumerate(data_loader):
        images = images.to(device)
        
        #create labels with are later used as input for BCE loss
        real_labels = torch.ones(batch_size).to(device)
        fake_labels = torch.zeros(batch_size).to(device)
        
        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #
        
        #compute BCE_loss using real images where BCE_loss(x,y) = -y*log(D(x)) - (1-y)*log(1-D(x))
        #second term of the loss is always zero for real cases because y=1 for real case
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        #compute BCEloss using fake images
        #First term of the loss is always zero for fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        #backprop and optimize
        d_loss = 0.5*d_loss_real + 0.5*d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        #train G to maximize log(D(G(z))) instead of minimizing log(1-D(G(z)))
        #use real label to keep the first term so loss = -log(D(G(z)))
        g_loss = criterion(outputs, real_labels)
        
        #back_prop
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
            
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
        
    # Save sampled images
    if (epoch<10 or epoch%5==0):
        fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
        save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))
    

/home/xiang/miniconda3/envs/ML/lib/python3.6/site-packages/torch/nn/functional.py:1474: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])) is deprecated. Please ensure they have the same size.
  "Please ensure they have the same size.".format(target.size(), input.size()))


Epoch [0/200], Step [50/600], d_loss: 0.7116, g_loss: 0.6335, D(x): 0.53, D(G(z)): 0.54
Epoch [0/200], Step [100/600], d_loss: 0.7025, g_loss: 0.6699, D(x): 0.51, D(G(z)): 0.52
Epoch [0/200], Step [150/600], d_loss: 0.6699, g_loss: 0.7060, D(x): 0.53, D(G(z)): 0.50
Epoch [0/200], Step [200/600], d_loss: 0.6687, g_loss: 0.7417, D(x): 0.51, D(G(z)): 0.48
Epoch [0/200], Step [250/600], d_loss: 0.6419, g_loss: 0.7560, D(x): 0.53, D(G(z)): 0.47
Epoch [0/200], Step [300/600], d_loss: 0.6087, g_loss: 0.7834, D(x): 0.56, D(G(z)): 0.46
Epoch [0/200], Step [350/600], d_loss: 0.5800, g_loss: 0.8620, D(x): 0.57, D(G(z)): 0.44
Epoch [0/200], Step [400/600], d_loss: 0.5266, g_loss: 0.8451, D(x): 0.63, D(G(z)): 0.44
Epoch [0/200], Step [450/600], d_loss: 0.5754, g_loss: 0.9228, D(x): 0.55, D(G(z)): 0.42
Epoch [0/200], Step [500/600], d_loss: 0.5910, g_loss: 0.8690, D(x): 0.56, D(G(z)): 0.44
Epoch [0/200], Step [550/600], d_loss: 0.5430, g_loss: 0.8797, D(x): 0.60, D(G(z)): 0.43
Epoch [0/200], Step [6

Epoch [7/200], Step [500/600], d_loss: 0.4464, g_loss: 1.4378, D(x): 0.69, D(G(z)): 0.35
Epoch [7/200], Step [550/600], d_loss: 0.3270, g_loss: 1.1083, D(x): 0.75, D(G(z)): 0.27
Epoch [7/200], Step [600/600], d_loss: 0.4079, g_loss: 1.3862, D(x): 0.64, D(G(z)): 0.27
Epoch [8/200], Step [50/600], d_loss: 0.4511, g_loss: 1.4440, D(x): 0.66, D(G(z)): 0.34
Epoch [8/200], Step [100/600], d_loss: 0.3641, g_loss: 1.4439, D(x): 0.74, D(G(z)): 0.31
Epoch [8/200], Step [150/600], d_loss: 0.4307, g_loss: 1.0790, D(x): 0.61, D(G(z)): 0.25
Epoch [8/200], Step [200/600], d_loss: 0.4547, g_loss: 1.2822, D(x): 0.70, D(G(z)): 0.37
Epoch [8/200], Step [250/600], d_loss: 0.3895, g_loss: 1.2122, D(x): 0.74, D(G(z)): 0.34
Epoch [8/200], Step [300/600], d_loss: 0.3972, g_loss: 1.0859, D(x): 0.69, D(G(z)): 0.29
Epoch [8/200], Step [350/600], d_loss: 0.4446, g_loss: 1.2394, D(x): 0.75, D(G(z)): 0.41
Epoch [8/200], Step [400/600], d_loss: 0.4246, g_loss: 1.3953, D(x): 0.66, D(G(z)): 0.31
Epoch [8/200], Step [4

Epoch [15/200], Step [300/600], d_loss: 0.4636, g_loss: 1.2832, D(x): 0.65, D(G(z)): 0.31
Epoch [15/200], Step [350/600], d_loss: 0.3150, g_loss: 1.7728, D(x): 0.74, D(G(z)): 0.24
Epoch [15/200], Step [400/600], d_loss: 0.3829, g_loss: 1.6396, D(x): 0.67, D(G(z)): 0.24
Epoch [15/200], Step [450/600], d_loss: 0.3322, g_loss: 1.6252, D(x): 0.78, D(G(z)): 0.29
Epoch [15/200], Step [500/600], d_loss: 0.3258, g_loss: 1.4814, D(x): 0.81, D(G(z)): 0.30
Epoch [15/200], Step [550/600], d_loss: 0.4514, g_loss: 1.2915, D(x): 0.67, D(G(z)): 0.33
Epoch [15/200], Step [600/600], d_loss: 0.4621, g_loss: 1.7505, D(x): 0.64, D(G(z)): 0.29
Epoch [16/200], Step [50/600], d_loss: 0.3875, g_loss: 1.5267, D(x): 0.71, D(G(z)): 0.30
Epoch [16/200], Step [100/600], d_loss: 0.4011, g_loss: 1.0978, D(x): 0.73, D(G(z)): 0.33
Epoch [16/200], Step [150/600], d_loss: 0.3217, g_loss: 1.6090, D(x): 0.73, D(G(z)): 0.22
Epoch [16/200], Step [200/600], d_loss: 0.3390, g_loss: 1.8616, D(x): 0.74, D(G(z)): 0.25
Epoch [16/2

Epoch [23/200], Step [100/600], d_loss: 0.3055, g_loss: 2.0114, D(x): 0.75, D(G(z)): 0.23
Epoch [23/200], Step [150/600], d_loss: 0.3201, g_loss: 2.2189, D(x): 0.77, D(G(z)): 0.27
Epoch [23/200], Step [200/600], d_loss: 0.4085, g_loss: 1.6345, D(x): 0.67, D(G(z)): 0.28
Epoch [23/200], Step [250/600], d_loss: 0.3990, g_loss: 1.5666, D(x): 0.76, D(G(z)): 0.34
Epoch [23/200], Step [300/600], d_loss: 0.4439, g_loss: 1.6351, D(x): 0.69, D(G(z)): 0.34
Epoch [23/200], Step [350/600], d_loss: 0.4558, g_loss: 1.7495, D(x): 0.69, D(G(z)): 0.33
Epoch [23/200], Step [400/600], d_loss: 0.4126, g_loss: 1.8746, D(x): 0.73, D(G(z)): 0.33
Epoch [23/200], Step [450/600], d_loss: 0.4691, g_loss: 1.7159, D(x): 0.74, D(G(z)): 0.39
Epoch [23/200], Step [500/600], d_loss: 0.3807, g_loss: 1.9158, D(x): 0.71, D(G(z)): 0.28
Epoch [23/200], Step [550/600], d_loss: 0.3305, g_loss: 1.1407, D(x): 0.74, D(G(z)): 0.26
Epoch [23/200], Step [600/600], d_loss: 0.3277, g_loss: 2.1242, D(x): 0.78, D(G(z)): 0.29
Epoch [24/

...

Epoch [168/200], Step [50/600], d_loss: 0.2629, g_loss: 2.6854, D(x): 0.73, D(G(z)): 0.09
Epoch [168/200], Step [100/600], d_loss: 0.1898, g_loss: 3.7824, D(x): 0.78, D(G(z)): 0.06
Epoch [168/200], Step [150/600], d_loss: 0.1270, g_loss: 2.4464, D(x): 0.94, D(G(z)): 0.14
Epoch [168/200], Step [200/600], d_loss: 0.2920, g_loss: 4.0218, D(x): 0.88, D(G(z)): 0.28
Epoch [168/200], Step [250/600], d_loss: 0.1110, g_loss: 3.5454, D(x): 0.92, D(G(z)): 0.09
Epoch [168/200], Step [300/600], d_loss: 0.1380, g_loss: 2.6210, D(x): 0.88, D(G(z)): 0.10
Epoch [168/200], Step [350/600], d_loss: 0.2444, g_loss: 3.3020, D(x): 0.85, D(G(z)): 0.22
Epoch [168/200], Step [400/600], d_loss: 0.1753, g_loss: 2.3075, D(x): 0.89, D(G(z)): 0.17
Epoch [168/200], Step [450/600], d_loss: 0.1262, g_loss: 2.7887, D(x): 0.92, D(G(z)): 0.14
Epoch [168/200], Step [500/600], d_loss: 0.1263, g_loss: 2.1656, D(x): 0.92, D(G(z)): 0.12
Epoch [168/200], Step [550/600], d_loss: 0.1686, g_loss: 2.5124, D(x): 0.94, D(G(z)): 0.18


Epoch [175/200], Step [400/600], d_loss: 0.1576, g_loss: 4.2184, D(x): 0.86, D(G(z)): 0.10
Epoch [175/200], Step [450/600], d_loss: 0.1363, g_loss: 2.7525, D(x): 0.86, D(G(z)): 0.07
Epoch [175/200], Step [500/600], d_loss: 0.1307, g_loss: 2.8827, D(x): 0.86, D(G(z)): 0.07
Epoch [175/200], Step [550/600], d_loss: 0.1116, g_loss: 3.1553, D(x): 0.87, D(G(z)): 0.04
Epoch [175/200], Step [600/600], d_loss: 0.1492, g_loss: 2.5523, D(x): 0.81, D(G(z)): 0.05
Epoch [176/200], Step [50/600], d_loss: 0.2162, g_loss: 1.7262, D(x): 0.99, D(G(z)): 0.28
Epoch [176/200], Step [100/600], d_loss: 0.1723, g_loss: 3.0582, D(x): 0.78, D(G(z)): 0.05
Epoch [176/200], Step [150/600], d_loss: 0.1017, g_loss: 1.8121, D(x): 0.90, D(G(z)): 0.07
Epoch [176/200], Step [200/600], d_loss: 0.0832, g_loss: 3.4496, D(x): 0.95, D(G(z)): 0.08
Epoch [176/200], Step [250/600], d_loss: 0.1679, g_loss: 4.3664, D(x): 0.83, D(G(z)): 0.10
Epoch [176/200], Step [300/600], d_loss: 0.2192, g_loss: 3.5507, D(x): 0.77, D(G(z)): 0.08


Epoch [183/200], Step [150/600], d_loss: 0.1075, g_loss: 2.5204, D(x): 0.93, D(G(z)): 0.11
Epoch [183/200], Step [200/600], d_loss: 0.1173, g_loss: 4.0754, D(x): 0.92, D(G(z)): 0.09
Epoch [183/200], Step [250/600], d_loss: 0.1799, g_loss: 3.3720, D(x): 0.78, D(G(z)): 0.05
Epoch [183/200], Step [300/600], d_loss: 0.0834, g_loss: 2.1015, D(x): 0.95, D(G(z)): 0.09
Epoch [183/200], Step [350/600], d_loss: 0.2212, g_loss: 2.3926, D(x): 0.96, D(G(z)): 0.28
Epoch [183/200], Step [400/600], d_loss: 0.1056, g_loss: 3.6844, D(x): 0.85, D(G(z)): 0.02
Epoch [183/200], Step [450/600], d_loss: 0.1117, g_loss: 2.3110, D(x): 0.94, D(G(z)): 0.13
Epoch [183/200], Step [500/600], d_loss: 0.0724, g_loss: 3.1675, D(x): 0.93, D(G(z)): 0.05
Epoch [183/200], Step [550/600], d_loss: 0.1154, g_loss: 3.1441, D(x): 0.91, D(G(z)): 0.10
Epoch [183/200], Step [600/600], d_loss: 0.5312, g_loss: 3.6186, D(x): 0.48, D(G(z)): 0.06
Epoch [184/200], Step [50/600], d_loss: 0.0898, g_loss: 3.3703, D(x): 0.89, D(G(z)): 0.04


Epoch [190/200], Step [500/600], d_loss: 0.1388, g_loss: 4.3397, D(x): 0.87, D(G(z)): 0.09
Epoch [190/200], Step [550/600], d_loss: 0.0806, g_loss: 3.4040, D(x): 0.96, D(G(z)): 0.09
Epoch [190/200], Step [600/600], d_loss: 0.0890, g_loss: 3.9659, D(x): 0.90, D(G(z)): 0.06
Epoch [191/200], Step [50/600], d_loss: 0.0784, g_loss: 3.6133, D(x): 0.90, D(G(z)): 0.03
Epoch [191/200], Step [100/600], d_loss: 0.1239, g_loss: 1.9738, D(x): 0.96, D(G(z)): 0.16
Epoch [191/200], Step [150/600], d_loss: 0.0609, g_loss: 4.0603, D(x): 0.95, D(G(z)): 0.06
Epoch [191/200], Step [200/600], d_loss: 0.1726, g_loss: 4.3428, D(x): 0.88, D(G(z)): 0.14
Epoch [191/200], Step [250/600], d_loss: 0.1859, g_loss: 3.4246, D(x): 0.87, D(G(z)): 0.13
Epoch [191/200], Step [300/600], d_loss: 0.1670, g_loss: 4.2335, D(x): 0.79, D(G(z)): 0.05
Epoch [191/200], Step [350/600], d_loss: 0.0782, g_loss: 3.8292, D(x): 0.93, D(G(z)): 0.07
Epoch [191/200], Step [400/600], d_loss: 0.1171, g_loss: 3.5940, D(x): 0.87, D(G(z)): 0.05


Epoch [198/200], Step [250/600], d_loss: 0.0913, g_loss: 2.7984, D(x): 0.94, D(G(z)): 0.10
Epoch [198/200], Step [300/600], d_loss: 0.1168, g_loss: 2.4697, D(x): 0.95, D(G(z)): 0.14
Epoch [198/200], Step [350/600], d_loss: 0.2307, g_loss: 2.4409, D(x): 0.98, D(G(z)): 0.29
Epoch [198/200], Step [400/600], d_loss: 0.0836, g_loss: 3.9745, D(x): 0.92, D(G(z)): 0.06
Epoch [198/200], Step [450/600], d_loss: 0.1334, g_loss: 3.7699, D(x): 0.83, D(G(z)): 0.04
Epoch [198/200], Step [500/600], d_loss: 0.1430, g_loss: 3.7961, D(x): 0.87, D(G(z)): 0.11
Epoch [198/200], Step [550/600], d_loss: 0.1528, g_loss: 1.7132, D(x): 0.91, D(G(z)): 0.15
Epoch [198/200], Step [600/600], d_loss: 0.1824, g_loss: 3.1998, D(x): 0.94, D(G(z)): 0.21
Epoch [199/200], Step [50/600], d_loss: 0.0761, g_loss: 4.4996, D(x): 0.89, D(G(z)): 0.02
Epoch [199/200], Step [100/600], d_loss: 0.1718, g_loss: 3.0441, D(x): 0.81, D(G(z)): 0.07
Epoch [199/200], Step [150/600], d_loss: 0.1611, g_loss: 4.1531, D(x): 0.81, D(G(z)): 0.05


In [24]:
G(x)

tensor([[[[-0.8798, -0.9389, -0.9630,  ..., -0.9627, -0.9627, -0.9551],
          [-0.9503, -0.9661, -0.9560,  ..., -0.9542, -0.9546, -0.9558],
          [-0.9680, -0.9602, -0.9556,  ..., -0.9505, -0.9504, -0.9558],
          ...,
          [-0.9657, -0.9530, -0.9557,  ..., -0.9457, -0.9478, -0.9557],
          [-0.9597, -0.9530, -0.9477,  ..., -0.9508, -0.9231, -0.9258],
          [-0.9570, -0.9538, -0.9552,  ..., -0.9570, -0.9254, -0.8154]]],


        [[[-0.9314, -0.9482, -0.9606,  ..., -0.9599, -0.9607, -0.9551],
          [-0.9612, -0.9632, -0.9546,  ..., -0.9531, -0.9535, -0.9558],
          [-0.9682, -0.9617, -0.9586,  ..., -0.9506, -0.9486, -0.9545],
          ...,
          [-0.9737, -0.9704, -0.9640,  ..., -0.9396, -0.8987, -0.8624],
          [-0.9633, -0.9501, -0.9513,  ..., -0.9500, -0.9289, -0.8951],
          [-0.9352, -0.8700, -0.9605,  ..., -0.9584, -0.9282, -0.7866]]],


        [[[-0.7739, -0.7704, -0.8813,  ..., -0.9688, -0.9688, -0.9563],
          [-0.8523, -0.548

In [25]:
z = torch.randn(batch_size, latent_size,1,1).to(device)
fake_images = G(z)

In [26]:

# Save the model checkpoints 
torch.save(G.state_dict(), 'Gcnn_latent2.ckpt')
torch.save(D.state_dict(), 'Dcnn_latent2.ckpt')